# Achitecture 1 : "Learning Image Matching by Simply Watching Video"

In [1]:
import numpy as np
import tensorflow as tf
import os
import time
import cv2
import sys
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import os

import numpy as np
import tensorflow as tf

import reader

print(sys.version)

ImportError: No module named cv2

## Parameters

In [ ]:
_conv_layers = [6,96,96,128,128,128]
_activation = 'sigmoid'

_batch_size = 32 
_learning_rate = 0.05
_epochs = 20
_step_test = 100
_step_viz = 100

_dataset = "ptb.train.txt"

_data_folder = "data/"
_train_folder = "train/"
_previously_trained = True

if not os.path.exists(_train_folder):
    os.makedirs(_train_folder)
    print("Directory created :", _train_folder)

In [ ]:
def activation(x,name=None):
    if _activation == 'sigmoid':
        return tf.nn.sigmoid(x,name)
    if _activation == 'relu':
        return tf.nn.relu(x,name)
  
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name='weights')

def bias_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name='bias')

## Create training batch

In [ ]:
def _read_words(filename):
    with tf.gfile.GFile(filename, "r") as f:
        return f.read().replace("\n", "<eos>").split()


def _build_vocab(filename):
    data = _read_words(filename)

    counter = collections.Counter(data)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

    words, _ = list(zip(*count_pairs))
    word_to_id = dict(zip(words, range(len(words))))

    return word_to_id


def _file_to_word_ids(filename, word_to_id):
    data = _read_words(filename)
    return [word_to_id[word] for word in data]


def ptb_raw_data(data_path=None):
    """Load PTB raw data from data directory "data_path".

    Reads PTB text files, converts strings to integer ids,
    and performs mini-batching of the inputs.

    The PTB dataset comes from Tomas Mikolov's webpage:

    http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz

    Args:
    data_path: string path to the directory where simple-examples.tgz has
    been extracted.

    Returns:
    tuple (train_data, valid_data, test_data, vocabulary)
    where each of the data objects can be passed to PTBIterator.
    """

    train_path = os.path.join(data_path, "ptb.train.txt")
    valid_path = os.path.join(data_path, "ptb.valid.txt")
    test_path = os.path.join(data_path, "ptb.test.txt")

    word_to_id = _build_vocab(train_path)
    train_data = _file_to_word_ids(train_path, word_to_id)
    valid_data = _file_to_word_ids(valid_path, word_to_id)
    test_data = _file_to_word_ids(test_path, word_to_id)
    vocabulary = len(word_to_id)
    return train_data, valid_data, test_data, vocabulary


def ptb_iterator(raw_data, batch_size, num_steps):
    """Iterate on the raw PTB data.

    This generates batch_size pointers into the raw PTB data, and allows
    minibatch iteration along these pointers.

    Args:
    raw_data: one of the raw data outputs from ptb_raw_data.
    batch_size: int, the batch size.
    num_steps: int, the number of unrolls.

    Yields:
    Pairs of the batched data, each a matrix of shape [batch_size, num_steps].
    The second element of the tuple is the same data time-shifted to the
    right by one.

    Raises:
    ValueError: if batch_size or num_steps are too high.
    """
    raw_data = np.array(raw_data, dtype=np.int32)

    data_len = len(raw_data)
    batch_len = data_len // batch_size
    data = np.zeros([batch_size, batch_len], dtype=np.int32)
    for i in range(batch_size):
        data[i] = raw_data[batch_len * i:batch_len * (i + 1)]

    epoch_size = (batch_len - 1) // num_steps

    if epoch_size == 0:
        raise ValueError("epoch_size == 0, decrease batch_size or num_steps")

    for i in range(epoch_size):
        x = data[:, i*num_steps:(i+1)*num_steps]
        y = data[:, i*num_steps+1:(i+1)*num_steps+1]
        yield (x, y)

## Convolutional Network graph

In [ ]:
raw_data = ptb_raw_data(_data_folder)
train_data, valid_data, test_data, _ = raw_data

batch_size = 32
num_steps = 50
vocab_size = 10000

sess = tf.InteractiveSession()

xi = tf.placeholder("float32",[None,num_steps],name='x_input')
yi = tf.placeholder("float32",[None,num_steps],name='y-truth')
x = tf.expand_dims(xi,axis=2)
y = yi#tf.expand_dims(yi,axis=2)

#CONVOLUTIONAL ENCODER


with tf.variable_scope('conv1') as scope:
    W1 = weight_variable([3,1,100])
    b1 = bias_variable([100])       
    preacti1 = tf.nn.bias_add(tf.nn.conv1d(x,W1,1,padding='SAME'),b1)
    conv1 = activation(preacti1,name=scope.name)

with tf.variable_scope('fc') as scope:
    Wfc = weight_variable([num_steps*100,num_steps])
    bfc = bias_variable([num_steps])
    h_flat = tf.reshape(conv1, [-1,num_steps*100])
    out = tf.nn.bias_add(tf.matmul(h_flat,Wfc),bfc)
    output = tf.reshape(out,(-1,num_steps,1))
    
with tf.variable_scope('cross_entropy') as scope:    
    
    logits = tf.nn.softmax(out)
    
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y* tf.log(logits), reduction_indices=[1]))

   


## Loss and metrics

In [ ]:
with tf.name_scope('loss') as scope:
    cost = tf.exp(cross_entropy)
    tf.summary.scalar('loss',cost)

global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(_learning_rate, global_step,10000, 0.99, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost,global_step=global_step)

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter(_train_folder+'/cnn', sess.graph)
saver = tf.train.Saver()

#initialization
init = tf.global_variables_initializer()

#launch graph
sess.run(init)


## Launch Training

In [ ]:
def run_epoch(session, data,voc_size,verbose=False):
    """Runs the model on the given data."""
    epoch_size = ((len(data) // batch_size) - 1) // num_steps
    start_time = time.time()
    costs = 0.0
    iters = 0

    for step, (xin, yin) in enumerate(reader.ptb_iterator(data, batch_size,num_steps)):
        c, _ = session.run([cost,optimizer],{xi: xin,yi: yin})
        costs += c
        iters += num_steps

        if verbose and step % (epoch_size // 10) == 10:
            print("%.3f perplexity: %.3f speed: %.0f wps" %(step * 1.0 / epoch_size, costs,iters * batch_size / (time.time() - start_time)))
    return costs

raw_data = reader.ptb_raw_data(_data_folder)
train_data, valid_data, test_data, voc_size = raw_data

for i in range(_epochs):
    train_perplexity = run_epoch(sess, train_data,voc_size,verbose=True)
    print("Epoch: %d Train Perplexity: %.3f" % (i + 1, train_perplexity))
    valid_perplexity = run_epoch(sess, valid_data,voc_size,verbose=True)
    print("Epoch: %d Valid Perplexity: %.3f" % (i + 1, valid_perplexity))

test_perplexity = run_epoch(sess, test_data,voc_size)
print("Test Perplexity: %.3f" % test_perplexity)